In [2]:
import keras
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout, Reshape
from keras.optimizers import SGD
from keras.callbacks import Callback
from keras.datasets import mnist

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Utils

In [3]:
def plotHistory(loss, acc):
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 2, 1)

    plt.plot(loss)
    plt.title('loss through training')
    plt.ylabel('loss')
    plt.xlabel('batch')

    plt.subplot(1, 2, 2)
    plt.plot(acc, 'g')
    plt.title('accuracy through training')
    plt.ylabel('accuracy')
    plt.xlabel('batch')

# Dataset

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float16') / 255.
x_test = x_test.astype('float16') / 255.

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print("training data: {}".format(x_train.shape))
print("testing data: {}".format(x_test.shape))

training data: (60000, 28, 28, 1)
testing data: (10000, 28, 28, 1)


# Network

In [8]:
model = Sequential([
    Convolution2D(16, kernel_size=5, padding="same", activation="relu", input_shape=(28, 28, 1,)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(10, activation="softmax"),
])

decoder = Sequential([
    Dense(196, activation="relu", input_shape=(10,)),
    Dense(784, activation="relu"),
    Reshape((28, 28, 1))
])

autoencoder = Sequential([
    model,
    decoder
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                31370     
Total params: 31,786
Trainable params: 31,786
Non-trainable params: 0
_________________________________________________________________


# Hyper Parameters

In [9]:
sgd = SGD(lr=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

autoencoder.compile(loss='mean_squared_error',
                   optimizer=sgd)

# Training

In [10]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.acc = []

    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))

In [11]:
history = LossHistory()
# model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=128, callbacks=[history])
autoencoder.fit(x_train, x_train, validation_data=(x_test, x_test), epochs=1, batch_size=64, callbacks=[history])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
32960/60000 [===============>..............] - ETA: 36s - loss: 0.1060

KeyboardInterrupt: 

In [ ]:
plotHistory(history.loss, history.acc)

# Test

In [ ]:
n_image = 5

display_index = np.random.choice(x_test.shape[0], n_image)
images = np.array([x_train[i] for i in display_index])
id_class = np.argmax(model.predict(images), axis=1)

plt.figure(figsize=(20, 4))

for i in range(n_image):
    ax = plt.subplot(1, n_image, i + 1)
    plt.imshow(images[i].reshape(28, 28), cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax.set_title("{0}".format(id_class[i]))
    